# Imports and functions

In [6]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from numpy import random
from gurobipy import *
from scipy.spatial.distance import euclidean
from scipy.stats import truncnorm
import pickle

path = "C:/Users/a.rojasa55/OneDrive - Universidad de los Andes/Documentos/MOPTA-23/Data/"

stations = pd.read_csv(path+"Definite Stations.csv",index_col=[0])
vehicles = pd.read_csv(path+"Definite Vehicles.csv",index_col=[0])

cd = 0.041
cw = 0.0388

In [2]:
loc, scale, min_v, max_v= 100, 50, 20, 250
aa, b = (min_v - loc) / scale, (max_v - loc) / scale

def generate_distances(vehicles,stations):
    distances = dict()
    for i in vehicles.index:
        for st in stations.index:    
            distances[st,i] = euclidean([vehicles["0"][i], vehicles["1"][i]],[stations["0"][st], stations["1"][st]])
    return distances

def generate_feasible_instance(stations,distances,sc,charging_rate,driving_speed):
    flag = True
    ii = -1
    while flag:
        ii += 1
        flag = False
        total_v = 0
        ranges, realized = dict(), dict()

        for i in vehicles.index:
            feas = False
            rr = truncnorm.rvs(a = aa, b = b, loc = loc, scale = scale, size = 1)
            ranges[i] = rr[0]

            prob = np.exp(-0.012**2*(rr-20)**2)
            realization = random.choice([True,False], p = [prob[0], 1-prob[0]])
            realized[i] = realization

            if realization == True:
                total_v += 1
                for st in stations.index:    
                    if distances[st,i] <= rr:
                        feas = True; break

                if not feas:
                    flag = True
                    print(f'\tAttempt n. {ii} failed at vehicle {i}')
                    break
    
    K = [k for k in vehicles.index if realized[k]]
    file = open(path+f"K/K_sc{sc}", "wb")
    pickle.dump(K,file); file.close()

    r = {k:ranges[k] for k in K}
    file = open(path+f"r/r_sc{sc}", "wb")
    pickle.dump(r,file); file.close()

    tp = {k:random.choice([0,1,2],p=[1/3,1/3,1/3]) for k in K}
    file = open(path + f"/tp/tp_sc{sc}", "wb")
    pickle.dump(tp,file); file.close()

    S_k = {k:[s for s in stations.index if distances[s,k]<=r[k]] for k in K}
    K_s = {s:[k for k in K if s in S_k[k]] for s in stations.index}

    file = open(path + f'/S_k/Sk_sc{sc}', 'wb')
    pickle.dump(S_k, file)
    file.close()

    file = open(path + f'/K_s/Ks_sc{sc}', 'wb')
    pickle.dump(K_s, file)
    file.close()

    S_k10 = dict()
    for k in K:
        dist = {st:distances[st,k] for st in S_k[k]}
        S_k10[k] = [st for st in sorted(dist, key=dist.get, reverse=False)[:min(10,len(S_k[k]))]]
    
    K_s10 = {s:[k for k in K_s[s] if s in S_k10[k]] for s in stations.index}

    file = open(path + f'/S_k10/Sk_sc{sc}', 'wb')
    pickle.dump(S_k10, file)
    file.close()

    file = open(path + f'/K_s10/Ks_sc{sc}', 'wb')
    pickle.dump(K_s10, file)
    file.close()

    p = {(k,s):250-(r[k]-distances[s,k]) for s in stations.index for k in K_s10[s]}
    t = {(k,s):p[k,s]/charging_rate  for s in stations.index for k in K_s10[s]}
    a = {(k,s):distances[s,k]/driving_speed+2*tp[k] for s in stations.index for k in K_s10[s]}

    file = open(path + f'/p/p_{sc}', 'wb')
    pickle.dump(p, file)
    file.close()

    file = open(path + f'/a t/at_{sc}', 'wb')
    pickle.dump((a,t), file)
    file.close()

    print(f'\tAttempt no. {ii} succesful - {total_v} realized EV')

In [5]:
S = stations.index
file = open(path+"S","wb")
pickle.dump(S,file)

# Data processing

In [3]:
d_matrix = generate_distances(vehicles,stations)

In [4]:
random.seed(0)

charging_rate = 75*12/7
driving_speed = 75

for sc in range(25):
    print(f'\nGENERATING INSTANCE No. {sc}')
    generate_feasible_instance(stations,d_matrix,sc,charging_rate,driving_speed)


GENERATING INSTANCE No. 0
	Attempt n. 0 failed at vehicle 4298
	Attempt n. 1 failed at vehicle 4298
	Attempt n. 2 failed at vehicle 8601
	Attempt n. 3 failed at vehicle 3181
	Attempt no. 4 succesful - 4591 realized EV

GENERATING INSTANCE No. 1
	Attempt no. 0 succesful - 4521 realized EV

GENERATING INSTANCE No. 2
	Attempt no. 0 succesful - 4411 realized EV

GENERATING INSTANCE No. 3
	Attempt n. 0 failed at vehicle 4290
	Attempt n. 1 failed at vehicle 8614
	Attempt n. 2 failed at vehicle 3206
	Attempt n. 3 failed at vehicle 9680
	Attempt n. 4 failed at vehicle 5369
	Attempt no. 5 succesful - 4441 realized EV

GENERATING INSTANCE No. 4
	Attempt n. 0 failed at vehicle 1007
	Attempt no. 1 succesful - 4570 realized EV

GENERATING INSTANCE No. 5
	Attempt n. 0 failed at vehicle 7522
	Attempt no. 1 succesful - 4616 realized EV

GENERATING INSTANCE No. 6
	Attempt no. 0 succesful - 4633 realized EV

GENERATING INSTANCE No. 7
	Attempt no. 0 succesful - 4569 realized EV

GENERATING INSTANCE No. 

In [17]:
K_s = {}; p = {}
for sc in range(25):
    file = open(path+f"K_s10/Ks_sc{sc}", "rb")
    K_s[sc] = pickle.load(file); file.close()

    file = open(path+f"p/p_{sc}", "rb")
    p[sc] = pickle.load(file); file.close()


c_s = {}
for s in stations.index:
    exp_cost = 0
    for sc in range(25):
        if len(K_s[sc][s]) == 0:
            exp_cost += 250*(cd+cw)
        else:
            exp_cost += sum(cd*d_matrix[s,k]+cw*p[sc][k,s] for k in K_s[sc][s])/len(K_s[sc][s])
    exp_cost /= 25
    c_s[s] = exp_cost

file = open(path+"c_s","wb")
pickle.dump(c_s,file); file.close()

In [13]:
K_s[0]

{1: [35,
  65,
  82,
  87,
  101,
  131,
  138,
  145,
  156,
  157,
  167,
  178,
  212,
  227,
  1082,
  1109,
  1114,
  1164,
  1166,
  1235,
  1246,
  1253,
  1257,
  1273,
  1284,
  1306,
  1317,
  2161,
  2187,
  2198,
  2243,
  2259,
  2289,
  2296,
  2336,
  2363,
  2372,
  2385,
  3240,
  3267,
  3277,
  3296,
  3338,
  3375,
  3404,
  3411,
  3451,
  3455,
  3464,
  3475,
  4345,
  4375,
  4403,
  4423,
  4447,
  4461,
  4483,
  4490,
  4494,
  4543,
  5424,
  5430,
  5435,
  5480,
  5482,
  5502,
  5533,
  5540,
  5562,
  5569,
  5589,
  5600,
  5607,
  5613,
  6484,
  6504,
  6509,
  6514,
  6533,
  6559,
  6561,
  6575,
  6612,
  6652,
  6668,
  6686,
  6688,
  6692,
  7582,
  7588,
  7593,
  7612,
  7635,
  7691,
  7710,
  7720,
  7765,
  7767,
  7771,
  7780,
  7791,
  8635,
  8642,
  8667,
  8672,
  8697,
  8719,
  8739,
  8763,
  8777,
  8789,
  8799,
  8806,
  8810,
  8837,
  8846,
  9714,
  9721,
  9740,
  9746,
  9751,
  9849,
  9856,
  9867,
  9885,
  9916,
  9923]

# Data exploration

In [ ]:
fig, axes = plt.subplots(nrows=5,ncols=1,figsize=(20,25))



def violin_plots(ax,col,mi=None):
    
    cols = {0:["deepskyblue","navy"],1:["greenyellow","olivedrab"],2:["gold","darkgoldenrod"],3:["darkorange","saddlebrown"],4:["hotpink","mediumvioletred"]}

    violins = {}
    for sc in range(25):

        if mi is None: s = f"reachable stations {sc}"
        else: s = f"reachable stations {mi} miles {sc}"

        sts = [len(data.at[i,s]) for i in data.index if data.at[i,s] != ["NA"]]
        violins[sc] = ax.violinplot(dataset=sts,positions=[sc],quantiles=[0.25,0.5,0.75],widths=[1.2],showextrema=False)
    ax.set_xticklabels([f"{sc}" for sc in range(25)])
    if mi is None:
        ax.set_ylabel("Total reachable stations by vehicle")
    else:
        ax.set_ylabel(f"Reachable stations within {mi} miles")

    for vi in violins:
        violins[vi]["bodies"][0].set_facecolor(cols[col][0])
        violins[vi]["bodies"][0].set_edgecolor(cols[col][1])
        violins[vi]["cquantiles"].set_edgecolor(cols[col][1])
        for b in violins[vi]["bodies"]:
            m = np.mean(b.get_paths()[0].vertices[:,0])
            b.get_paths()[0].vertices[:, 0] = np.clip(b.get_paths()[0].vertices[:, 0], m, np.inf)
        for i in range(3):
            m = np.mean(violins[vi]["cquantiles"].get_paths()[i].vertices[:,0])
            violins[vi]["cquantiles"].get_paths()[i].vertices[:, 0] = np.clip(violins[vi]["cquantiles"].get_paths()[i].vertices[:, 0], m, np.inf)

    return ax


mis = [None,40,25,15,10]; maxs = [1250,650,500,350,250]
for i in range(5):
    axes[i] = violin_plots(axes[i],i,mis[i])
    axes[i].set_xlim(-0.5,25); axes[i].set_xticks(range(25))
    axes[i].set_ylim(0,maxs[i])



In [ ]:
fig, axes = plt.subplots(nrows=4,ncols=1,figsize=(20,25))

def violin_plots_stations(ax,col,mi=None):
    
    
    cols = {0:["deepskyblue","navy"],1:["greenyellow","olivedrab"],2:["gold","darkgoldenrod"],3:["darkorange","saddlebrown"],4:["hotpink","mediumvioletred"]}

    violins = {}
    for sc in range(25):

        s = f"K {mi} sc {sc}"
        sts = [len(stations.at[st,s]) for st in stations.index if stations.at[st,s]!=0]
        violins[sc] = ax.violinplot(dataset=sts,positions=[sc],quantiles=[0.25,0.5,0.75],widths=[1.2],showextrema=False)

    ax.set_xticklabels([f"{sc}" for sc in range(25)])
    if mi is None:
        ax.set_ylabel("Total reachable stations by vehicle")
    else:
        ax.set_ylabel(f"Possible vehicles closest-{mi}")

    for vi in violins:
        violins[vi]["bodies"][0].set_facecolor(cols[col][0])
        violins[vi]["bodies"][0].set_edgecolor(cols[col][1])
        violins[vi]["cquantiles"].set_edgecolor(cols[col][1])
        for b in violins[vi]["bodies"]:
            m = np.mean(b.get_paths()[0].vertices[:,0])
            b.get_paths()[0].vertices[:, 0] = np.clip(b.get_paths()[0].vertices[:, 0], m, np.inf)
        for i in range(3):
            m = np.mean(violins[vi]["cquantiles"].get_paths()[i].vertices[:,0])
            violins[vi]["cquantiles"].get_paths()[i].vertices[:, 0] = np.clip(violins[vi]["cquantiles"].get_paths()[i].vertices[:, 0], m, np.inf)

    return ax

mis = [40,25,15,10]; maxs = [1000,800,600,500]
for i in range(4):
    axes[i] = violin_plots_stations(axes[i],i,mis[i])
    axes[i].set_xlim(-0.5,25); axes[i].set_xticks(range(25))
    axes[i].set_ylim(0,maxs[i])

plt.savefig("./Violin stations.png",dpi=300)